<h2>Scraping Metacritic Scores

In [1]:
# Import Dependencies
import requests
from pprint import pprint
import pandas as pd
from bs4 import BeautifulSoup as bs
from decimal import Decimal
import re

In [3]:
# Try Metacritic Scrape for Top 100 Ranked Movies
url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc'
print(requests.get(url))

<Response [403]>


In [4]:
# Add headers to bypass 403 Forbidden Access
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Safari/605.1.15'}
response = requests.get(url, timeout=30, headers=headers)
print(response)

<Response [200]>


In [5]:
# Parse the HTML text using Beautiful Soup
soup = bs(response.text, 'html.parser')

# Find and append the metacritic score, rank, title, and short summary of each movie
top_100_metacritic_dicts = []
for td in soup.find_all('td', class_ = 'clamp-summary-wrap'):
    score = td.find('div', class_ = 'clamp-score-wrap').text.strip()
    rank = td.find('span', class_ = 'title numbered').text.strip()[:-1]
    title = td.find('a', class_ = 'title').text.strip()
    summary = td.find('div', class_ = 'summary').text.strip()
    top_100_metacritic_dicts.append({'score': score,
                                    'rank': rank,
                                    'title': title,
                                    'summary': summary})

print(len(top_100_metacritic_dicts))

100


<h2>Scraping Base Data (Top 100 Grossing Movies)

In [6]:
# Scrape the base data from the-numbers.com (for Top 100 Grossing Movies)
response = requests.get("https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time")
base_df_1 = pd.read_html(response.text)[0]
base_df_2 = pd.read_html(response.text)[1]
base_df = base_df_1.append(base_df_2).reset_index(drop=True)

# Make a copy of dataframe
base_df2 = base_df.copy()

# Loop through copy of dataframe and convert money text to decimal formatting
for index, row in base_df2.iterrows():
    # Convert to decimal
    money = re.match(r'\$\d+,\d+,\d+', row['DomesticBox Office']).group()
    base_df2 = base_df2.replace(row['DomesticBox Office'], Decimal(re.sub(r'[^\d.]', '', money)))
    money = re.match(r'\$\d+,\d+,\d+', row['InternationalBox Office']).group()
    base_df2 = base_df2.replace(row['InternationalBox Office'], Decimal(re.sub(r'[^\d.]', '', money)))
    money = re.match(r'\$\d+,\d+,\d+', row['WorldwideBox Office']).group()
    base_df2 = base_df2.replace(row['WorldwideBox Office'], Decimal(re.sub(r'[^\d.]', '', money)))
    
base_df2.head()

,Rank,Released,Movie,DomesticBox Office,InternationalBox Office,WorldwideBox Office
0,1,2015,Star Wars Ep. VII: The Force Awakens,936662225,1116648,2053311
1,2,2009,Avatar,760507625,2015837,2776345
2,3,2018,Black Panther,700059566,648300000,1348359
3,4,2018,Avengers: Infinity War,678815482,1369988,2048803
4,5,1997,Titanic,659363944,1548844,2208208


<h2>Scraping Base Data Movies (Top 100 Grossing Movies) from Metacritic

In [7]:
# Prepare Movie Title URLs for Metacritic calls
urls = []
for title in base_df.Movie:

    url_title = title.replace(' ', '-').replace(',', '').lower().replace('â', '')

    if ',' in url_title:
        url_title = url_title
    if 'â' in url_title:
        url_title = url_title

    # Replace ep. viii: with viii-- and all other cases found in top100 grossing movie titles
    if 'ep.-iv:' in url_title:
        url_title = url_title.replace('ep.-iv:', 'episode-iv--')
    elif 'ep.-iii:' in url_title:
        url_title = url_title.replace('ep.-iii:', 'episode-iii--')
    elif 'ep.-viii:' in url_title:
        url_title = url_title.replace('ep.-viii:', 'episode-viii--')
    elif 'ep.-vii:' in url_title:
        url_title = url_title.replace('ep.-vii:', 'episode-vii--')
    elif 'ep.-ii:' in url_title:
        url_title = url_title.replace('ep.-ii:', 'episode-ii--')
    elif 'ep.-i:' in url_title:
        url_title = url_title.replace('ep.-i:', 'episode-i--')
    elif 'part-ii' in url_title:
        url_title = url_title.replace('part-ii', 'part-2')
    elif 'part-i' in url_title:
        url_title = url_title.replace('part-i', 'part-1')

    # Now replace ':' and '.' with '' since we don't need it for matching anymore
    if ':' in url_title:
        url_title = url_title.replace(':', '')
    if '.' in url_title:
        url_title = url_title.replace('.', '')

    urls.append({'title': title, 'url': 'https://www.metacritic.com/movie/' + url_title})
    
len(urls)

100

In [9]:
urls

[{'title': 'Star Wars Ep. VII: The Force Awakens',
  'url': 'https://www.metacritic.com/movie/star-wars-episode-vii---the-force-awakens'},
 {'title': 'Avatar', 'url': 'https://www.metacritic.com/movie/avatar'},
 {'title': 'Black Panther',
  'url': 'https://www.metacritic.com/movie/black-panther'},
 {'title': 'Avengers: Infinity War',
  'url': 'https://www.metacritic.com/movie/avengers-infinity-war'},
 {'title': 'Titanic', 'url': 'https://www.metacritic.com/movie/titanic'},
 {'title': 'Jurassic World',
  'url': 'https://www.metacritic.com/movie/jurassic-world'},
 {'title': 'The Avengers',
  'url': 'https://www.metacritic.com/movie/the-avengers'},
 {'title': 'Star Wars Ep. VIII: The Last Jedi',
  'url': 'https://www.metacritic.com/movie/star-wars-episode-viii---the-last-jedi'},
 {'title': 'Incredibles 2',
  'url': 'https://www.metacritic.com/movie/incredibles-2'},
 {'title': 'The Dark Knight',
  'url': 'https://www.metacritic.com/movie/the-dark-knight'},
 {'title': 'Rogue One: A Star War

In [8]:
# Scraping Metacritic for each of the top 100 grossing movies
top_100_grossing_metacritic_dicts = []

for url in urls:

    # Parse the HTML text using Beautiful Soup
    soup = bs(requests.get(url['url'], timeout=30, headers=headers).text, 'html.parser')
    
    # Find and append the release date, metacritic critic(meta) and user scores, production studio, 
    # diretor, main cast, genres, and parental rating, and runtime of each movie
    
    # Release Date
    try:
        for span in soup.find_all('span', class_ = 'release_date'):
            release_date = span.find_all('span')[1].text.strip()
    except:
        release_date = ''
    # Scores
    try:
        #critic_score = soup.find('div', class_ = 'primary_baby_item oswald').text.strip().split('\n')[0]
        #critic_score = soup.find('div', class_ = 'score fl').text.strip()
        scores = []
        for a in soup.find_all('a', class_ = 'metascore_anchor'):
            scores.append(a.text.strip())
        critic_score = scores[0]
        user_score = scores[1]
    except:
        critic_score = ''
        user_score = ''
        
    # Production Studio
    try:
        studio = soup.find('span', class_ = 'distributor').text.strip()[:-2]
    except:
        studio = ''
        
    # Director
    try:
        director = soup.find('div', class_ = 'director').text.strip().split('\n')[1]
    except:
        director = ''
        
    # Main Cast
    try:
        main_cast = soup.find('div', class_ = 'summary_cast details_section').text.strip().split('\n')[2].replace('  ', '').split(',')
    except:
        main_cast = ''
    
    # Genres
    try:
        genres = soup.find('div', class_ = 'genres').text.strip()[11:].replace(' ', '').split(',')
    except:
        genres = ''
    
    # Parental Rating
    try:
        parental_rating = soup.find('div', class_ = 'rating').text.strip().split('\n')[3].replace('  ', '')
    except:
        parental_rating = ''

    # Runtime
    try:
        runtime = soup.find('div', class_ = 'runtime').text.strip()[9:]
    except:
        runtime = ''
    
    # Append data as a dictionary to list of dictionaries
    top_100_grossing_metacritic_dicts.append({'title': url['title'],
                                              'release_date': release_date,
                                              'critic_score': critic_score,
                                              'user_score': user_score,
                                              'studio': studio,
                                              'director': director,
                                              'main_cast': main_cast,
                                              'genres': genres,
                                              'parental_rating': parental_rating,
                                              'runtime': runtime
                                             })

pprint(top_100_grossing_metacritic_dicts)

KeyboardInterrupt: 